In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import numpy as np
import CleaningCars as hf
from sklearn.model_selection import train_test_split

import statsmodels.api as sm

In [2]:
def load_data():    
    csv_path = os.path.join('data', 'vehicles.csv')
    return pd.read_csv(csv_path)

df = load_data()

In [3]:
df = hf.remove_columns(df)

In [4]:
df.isnull().sum()

region               0
price                0
year              1050
manufacturer     18220
model             4846
condition       192940
cylinders       171140
fuel              3237
odometer         55303
title_status      2577
transmission      2442
drive           134188
type            112738
paint_color     140843
description         70
state                0
lat               7448
long              7448
posting_date        28
dtype: int64

In [5]:
df = hf.imputeManufacturer(df)
df = hf.dateToDatetime(df)
df = hf.basicImpute(df)
df = hf.imputeMissingByManufacturer(df, col='fuel')
df = hf.imputeMissingByManufacturer(df, col='transmission')
df = hf.imputeOdometerByYear(df)


In [6]:
df.isnull().sum()

region               0
price                0
year              1050
manufacturer     11915
model             4846
condition       192940
cylinders       171140
fuel               296
odometer            28
title_status      2577
transmission       155
drive           134188
type            112738
paint_color     140843
description          0
state                0
lat               7448
long              7448
posting_date         0
dtype: int64

In [7]:
    df = df[df['year'].notna()]

In [8]:
df.isnull().sum()

region               0
price                0
year                 0
manufacturer     11106
model             4817
condition       191892
cylinders       171094
fuel               140
odometer             0
title_status      2298
transmission       127
drive           133998
type            112633
paint_color     140740
description          0
state                0
lat               7420
long              7420
posting_date         0
dtype: int64

In [11]:
df = hf.color_clean(df, color_list=['white','black','silver'])
df = hf.drive_clean(df)
df = hf.transmission_clean(df)
df = hf.titlestatus_clean(df)
df = hf.cleanLocationFeatures(df)
print("One hot encodings done!")

One hot encodings done!


In [14]:
df.columns

Index(['price', 'year', 'manufacturer', 'model', 'condition', 'cylinders',
       'fuel', 'odometer', 'type', 'description', 'lat', 'long',
       'posting_date', 'color_black', 'color_other', 'color_silver',
       'color_white', 'drive_4wd', 'drive_fwd', 'drive_other', 'drive_rwd',
       'transmission_automatic', 'transmission_manual', 'transmission_other',
       'status_clean', 'status_lien', 'status_missing', 'status_parts only',
       'status_rebuilt', 'status_salvage', 'state_ak', 'state_al', 'state_ar',
       'state_az', 'state_ca', 'state_co', 'state_ct', 'state_dc', 'state_de',
       'state_fl', 'state_ga', 'state_hi', 'state_ia', 'state_id', 'state_il',
       'state_in', 'state_ks', 'state_ky', 'state_la', 'state_ma', 'state_md',
       'state_me', 'state_mi', 'state_mn', 'state_mo', 'state_ms', 'state_mt',
       'state_nc', 'state_nd', 'state_ne', 'state_nh', 'state_nj', 'state_nm',
       'state_nv', 'state_ny', 'state_oh', 'state_ok', 'state_or', 'state_pa',
       

In [35]:
pd.get_dummies(df["cylinders"])

,10 cylinders,12 cylinders,3 cylinders,4 cylinders,5 cylinders,6 cylinders,8 cylinders,missing,other
0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
458208,0,0,0,0,0,1,0,0,0
458209,0,0,0,1,0,0,0,0,0
458210,0,0,0,0,0,1,0,0,0
458211,0,0,0,0,0,0,1,0,0


In [36]:
df["cylinders"] = df["cylinders"].fillna("missing")

In [40]:
pd.get_dummies(df, columns = ["cylinders"]).drop(["cylinders_12 cylinders", "cylinders_10 cylinders", "cylinders_3 cylinders", "cylinders_5 cylinders", "cylinders_other"], axis = 1)

,price,year,manufacturer,model,condition,fuel,odometer,type,description,lat,...,state_va,state_vt,state_wa,state_wi,state_wv,state_wy,cylinders_4 cylinders,cylinders_6 cylinders,cylinders_8 cylinders,cylinders_missing
0,35990,2010.0,chevrolet,corvette grand sport,good,gas,32742.0,other,Carvana is the safer way to buy a car During t...,32.590000,...,0,0,0,0,0,0,0,0,1,0
1,7500,2014.0,hyundai,sonata,excellent,gas,93600.0,sedan,I'll move to another city and try to sell my c...,32.547500,...,0,0,0,0,0,0,1,0,0,0
2,4900,2006.0,bmw,x3 3.0i,good,gas,87046.0,SUV,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,32.616807,...,0,0,0,0,0,0,0,1,0,0
3,2000,1974.0,chevrolet,c-10,good,gas,190000.0,pickup,1974 chev. truck (LONG BED) NEW starter front ...,32.861600,...,0,0,0,0,0,0,1,0,0,0
4,19500,2005.0,ford,f350 lariat,excellent,diesel,116000.0,pickup,2005 Ford F350 Lariat (Bullet Proofed). This t...,32.547500,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458208,14995,2014.0,cadillac,srx,excellent,gas,83790.0,SUV,2014 Cadillac SRX All Wheel Drive Luxury Colle...,44.408867,...,0,0,0,1,0,0,0,1,0,0
458209,17895,2018.0,hyundai,tucson,excellent,gas,31888.0,SUV,2018 Hyundai Tucson SE 2.0 I4 AWD. Carfax ver...,44.408867,...,0,0,0,1,0,0,1,0,0,0
458210,17900,2014.0,cadillac,cts coupe awd,excellent,gas,53000.0,coupe,"53,000 miles. Cosmetically, in near mint condi...",44.784900,...,0,0,0,0,0,1,0,1,0,0
458211,1300,2008.0,jeep,grand cherokee,good,gas,164000.0,SUV,PRICE REDUCTION Turns out the engine is toast...,44.773500,...,0,0,0,0,0,1,0,0,1,0


In [ ]:
df_processed = hf.ultimateClean(df)

Cleaned !


In [ ]:
df_processed.shape

In [ ]:
df_processed.columns